#### Importing all the necessary packages

In [1]:
import os
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.utilities import GoogleSearchAPIWrapper
from langchain import OpenAI, ConversationChain
from dotenv import load_dotenv

#### Reading API Keys
- Keys are stored in .env file which is not public.
- Steps to create a .env file:
  - Create a separate file to store your environment variables. This file should have a name that starts with a dot (.), for example, ".env_file".
  - Open the .env_file in a text editor and add your environment variables in the following format
    - VARIABLE_NAME=variable_value
  - In your Python program, install the python-dotenv package
    - Note this will already be installed if you use poetry and .toml to install the dependencies.
- Reference to setup Google Search API Key and CSE ID:
  - https://python.langchain.com/en/latest/modules/agents/tools/examples/google_search.html


In [2]:
# Load the dot environment from the created file
load_dotenv(dotenv_path='../.env')

True

In [3]:
open_api_key = os.environ.get("OPENAI_API_KEY")
google_api_key = os.environ.get("GOOGLE_API_KEY")
google_cse_id = os.environ.get("GOOGLE_CSE_ID")

In [4]:
llm  = OpenAI(temperature = 0.9, openai_api_key=open_api_key) # Amount of randomness, higher is more random

In [5]:
text = "What are 5 vacation destination for someone who likes to eat pasta?"
print(llm(text))



1. Rome, Italy 
2. Tuscany, Italy 
3. Naples, Italy 
4. Bologna, Italy 
5. Milan, Italy


### Prompt Templates: Manage prompts for LLMs

In [6]:
prompt = PromptTemplate(
    input_variables=["food"],
    template = "What are 5 vacation destination for someone who likes to eat {food}?",
)

In [7]:
print(prompt.format(food="dessert"))

What are 5 vacation destination for someone who likes to eat dessert?


In [8]:
print(llm(prompt.format(food="dessert")))

 

1. New York City, USA: With a wide variety of delicious desserts, from cronuts to ice cream sandwiches, the city's vast selection of bakeries and restaurants offer something for every sweet tooth. 

2. Tokyo, Japan: From soft, creamy cheesecakes to traditional wagashi mochi, the Japanese capital has a plethora of delicious desserts to offer. 

3. Buenos Aires, Argentina: Home to the famous alfajores cookie sandwich, Buenos Aires offers a host of indulgent desserts, such as churros, dulce de leche and more. 

4. Paris, France: Whether you’re looking for a rich and decadent chocolate cake or the traditional French confection known as the Paris-Brest, the City of Lights offers an abundance of treats for dessert lovers. 

5. Hong Kong: From mango sago to egg tarts, the Chinese city boasts a unique combination of traditional Cantonese and modern desserts.


### Chains: Combine LLMs and prompts in multi-step workflows

In [9]:
prompt = PromptTemplate(
    input_variables=["food"],
    template = "What are 5 vacation destination for someone who likes to eat {food}?",
)

In [10]:
chain = LLMChain(llm=llm, prompt=prompt)

In [11]:
print(chain.run("fruit"))



1. Hawaii: an abundance of tropical fruits like mangoes, pineapples, and papayas can be found at local markets. 

2. South Africa: sample the unique flavors of indigenous fruits like wild figs, mongongo nuts, and baobab fruits. 

3. Sri Lanka: try exotic fruits like jackfruit, rambutan, and mangosteen during your visit. 

4. Germany: enjoy fresh apples, pears, and cherries straight from the orchards. 

5. Dominican Republic: indulge in succulent mangoes, guavas, and passion fruit during your trip.


# Agents: Dynamically call chains based on user input

In [12]:
search = GoogleSearchAPIWrapper()

In [13]:
search.run("Obama's first name?")

"STATE OF HAWAII. 1 Child's First Name. (Type or print). BARACK. CERTIFICATE OF LIVE BIRTH lb. Middle Name. HUSSE. HUSSEIN. FILE. NUMBER 151. Barack Hussein Obama II is an American former politician who served as the 44th president of the United States from 2009 to 2017. A member of the Democratic\xa0... His full name is Barack Hussein Obama II. Since the “II” is simply because he was named for his father, his last name is Obama. Jan 19, 2017 ... Jordan Barack Treasure, New York City, born in 2008 ... Jordan Barack Treasure made national news when he was the focus of a New York newspaper\xa0... 4 days ago ... Barack Obama, in full Barack Hussein Obama II, (born August 4, 1961, Honolulu, Hawaii, U.S.), 44th president of the United States (2009–17) and\xa0... Mar 22, 2008 ... Barry Obama decided that he didn't like his nickname. A few of his friends at Occidental College had already begun to call him Barack (his\xa0... When Barack Obama was elected president in 2008, he became the first 

In [14]:
tools = load_tools(['google-search','llm-math'], llm=llm)

In [15]:
agent = initialize_agent(tools=tools, llm=llm, agent='zero-shot-react-description', verbose=True)

In [17]:
agent.run("Who is the current leader of India? What is the largest prime number that is smaller than their age?")



> Entering new AgentExecutor chain...
 I should look for the leader's age to answer the second part of the question.
Action: Google Search
Action Input: Current leader of India
Observation: The President of India, Smt Droupadi Murmu presented National Panchayat Awards and inaugurating National Conference on Incentivization of Panchayats in New ... Droupadi Murmu is the 15th and current president, having taken office from 25 July 2022. President of India. Bhārat kē Rāṣṭrapati. Emblem of India.svg · Emblem ... Additionally, the directory provides contact information for Indian Affairs leadership.p. There are currently 574 Federally Recognized Tribes as of 01/28/2022. Jul 25, 2022 ... Droupadi Murmu, a tribal politician, was sworn in as India's new president on Monday. The 64-year-old former teacher comes from Odisha (Orissa) ... To do this, we depend on great leadership. And that takes great leadership development for our members. ... Third Vice President. Kolkata, India. Apr 4, 2019 .

'The current leader of India is Droupadi Murmu, and the largest prime number that is smaller than her age is 61.'

## Adding Memory and state to chains and agents

In [18]:
conversation = ConversationChain(llm=llm, verbose=True)

In [19]:
conversation.predict(input="Hi There!")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi There!
AI:

> Finished chain.


' Hi there! How can I help you?'

In [20]:
conversation.predict(input="Just having a conversation with an AI-agent")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi There!
AI:  Hi there! How can I help you?
Human: Just having a conversation with an AI-agent
AI:

> Finished chain.


" Nice to meet you! It's always fun to talk to new people. My name is AI-Agent, what's yours?"

In [21]:
conversation.predict(input="What was the first thing I said to you?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi There!
AI:  Hi there! How can I help you?
Human: Just having a conversation with an AI-agent
AI:  Nice to meet you! It's always fun to talk to new people. My name is AI-Agent, what's yours?
Human: What was the first thing I said to you?
AI:

> Finished chain.


' You said, "Hi there!"'